In [10]:
import os
import math

In [11]:
# '''
# @breif: function for debugging
# @param x: input
# @return y
# '''

# def fun3(x):
#     y = (1.0/3.0)*x**3 - 0.5*x**2 - x - 1.0
#     return y

# def func(x):
#     y = math.cos(x)
#     return y

# '''
# @breif: parabolic interpolation method
# @param x1, y1, x2, y2, x3, y3: three points
# @return p, q: x4 = x3 + p/q               
# '''

# # x3<x2, x1
# #x = (x1^2*(y2-y3)+x2^2*(y3-y1)+x3^2*(y1-y2))/(2*(x1*(y2-y3)+x2*(y3-y1)+x3*(y1-y2)))
# #which is invariant under the permutation of the three points
# def parabolic_interpolation(x1, y1, x2, y2, x3, y3):
#     p = (x2 - x3)**2 * (y3 - y1) + (x1 - x3)**2 * (y2 - y3)
#     q = 2.0 * ((x2 - x3) * (y3 - y1) + (x1 - x3) * (y2 - y3))
#     return p, q
    
# '''
# @breif: brent method in one iteration, in one iteration, only one new point is evaluated(fu)

# @param a, b: boundary of the interval
# @param v, fv: the old value of w
# @param w, fw: second least value
# @param x, fx: the least value
# @param fun_p: the function to be minimized
# @param dx: the last step length
# @param dxold: the last last step length

# @return fv, fw, fx: updated values
# @return a, b, v, w, x, dx, dxold: updated values
# @return u: last evaluated point
# '''

# #* here I updated the values of fv, fw, fx to save the computation time, only fu is evaluated in one iteration
# def brent(a, b, v, fv, w, fw, x, fx, fun_p, dx, dxold):
#     CGOLD = 0.381966013
#     xm = 0.5 * (a + b)

#     #perform parabolic interpolation using v, w, x (x the minimum point)
#     p, q = parabolic_interpolation(v, fv, w, fw, x, fx)

#     #Boolean variables for the safety of parabolic interpolation
#     #* qSafe: q is not zero
#     qSafe = True if (q != 0) else False

#     tempdx = p/q if qSafe else 0.0
#     #* paraSafe:
#     #* 1. q is not zero
#     #* 2. the new point is within the interval
#     #* 3. the new step length is less than half of the last last step length
#     paraSafe = True if (qSafe and (a < x+tempdx) and (x+tempdx < b) and (abs(tempdx) < 0.5*abs(dxold))) else False

#     #update the dx and store the last last step length for next iteration

#     if paraSafe:
#         dxold = dx
#         dx = tempdx
#     else:
#         if x < xm:
#             dxold = b - x
#         else:
#             dxold = a - x
#         dx = CGOLD * dxold

#     #! evaluate the new point
#     u = x + dx
#     fu = fun_p(u)

#     if (fu < fx):
#         #* update the boundary of interval
#         a = a if (u < x) else x
#         b = x if (u < x) else b
#         #* update the values of v, w, x
#         v = w
#         fv = fw
#         w = x
#         fw = fx
#         x = u
#         fx = fu
#     else:
#         #* update the boundary of interval
#         a = u if (u < x) else a
#         b = b if (u < x) else u

#         #* update the values of v, w, x
#         if (fu < fw) or (w == x):
#             v = w
#             fv = fw
#             w = u
#             fw = fu
#         elif (fu < fv) or (v == x) or (v == w):
#             v = u
#             fv = fu
    
#     return a, b, v, fv, w, fw, x, fx, dx, dxold

# '''
# @breif: brent method
# @param a0, b0: initial boundary of the interval
# @param xguess: initial guess of the minimum point
# @param fun_p: the function to be minimized
# @param tol: tolerance

# @return xmin: the minimum point
# @return icount: the number of iterations
# '''
# def brentMethod(a0, b0, xguess, fun_p, tol):
#     Maxloop = 100

#     icount = 0
#     #! evaluate the function at the initial guess
#     y0 = fun_p(xguess)
#     a, b, v, fv, w, fw, x, fx, dx, dxold = brent(a0, b0, xguess, y0, xguess, y0, xguess, y0, fun_p, 0.0, 0.0)

#     print(f"x:\t{x:.6f}\ty:\t{fx:.12f}")

#     for i in range(Maxloop):
#         a, b, v, fv, w, fw, x, fx, dx, dxold = brent(a, b, v, fv, w, fw, x, fx, fun_p, dx, dxold)

#         print(f"x:\t{x:.6f}\ty:\t{fx:.12f}")

#         icount += 1
#         if (abs(b - a) < tol):
#             xmin = 0.5 * (a + b)
#             break
            
#     return xmin, icount
        


In [14]:
# '''
# @breif: find the spin index in the gjf file
# by locating the second blank line index, then add 1

# @param filename: default filename is "template.gjf"

# @retrun iopIndex: the index of the iop line (start from 1)
# @return spinIndex: the index of the spin line (start from 1)
# '''
# def findSpinIndex(filename = "template.gjf"):

#     with open(filename, "r") as gjf:
#         icount = 0
#         blankCount = 0
        
#         for line in gjf:
#             icount += 1
#             if line == "\n":
#                 blankCount += 1
#             if blankCount == 2:
#                 spinIndex = icount + 1
#                 break
        
#         iopIndex = spinIndex - 4

#     return iopIndex, spinIndex

# '''
# @brief: generate gjf with N+1, N, N-1 spins in different IOp(3/107, 3/108) values (int(wpara*1000):05 + 0:05 format)

# @param Nspin: "charge spin" of N
# @param Naspin: "charge spin" of N+1
# @param Nmspin: "charge spin" of N-1
# @param wpara: the parameter for the IOp(3/107, 3/108) value
# @param filename: the template file name, default is "template.gjf"


# @return: void, three gjf files: N.gjf, N+1.gjf, N-1.gjf
# '''
# #! here I use f"{int(wpara*10000):05}" to convert the float to string, by adding 5 zeros
# #! it's not good, I suggest to use round(x,4) functon instead

# def g16input(Nspin, Naspin, Nmspin, wpara, filename = "template.gjf"):

#     iopIndex, spinIndex = findSpinIndex()

#     with open("N.gjf", "w") as Ngjf, open("N+1.gjf", "w") as Nagjf, open("N-1.gjf", "w") as Nmgjf:
#         with open(filename, "r") as gjf:
#             icount = 0
#             for line in gjf:
#                 icount += 1
#                 if icount == iopIndex:
#                     tempIop = line.strip()  #*remove the newline character
#                     wpara = round(wpara, 4)    #*round the float to 4 decimal places
#                     str_wpara = f"{int(wpara*10000):05}"   #*convert the float to string, by adding 5 zeros
#                     str_iop = str_wpara + f"{0:05}"
                    
#                     Ngjf.write(f"{tempIop} IOp(3/107={str_iop},3/108={str_iop})\n")
#                     Nagjf.write(f"{tempIop} IOp(3/107={str_iop},3/108={str_iop})\n")
#                     Nmgjf.write(f"{tempIop} IOp(3/107={str_iop},3/108={str_iop})\n")

#                 elif icount == spinIndex:
#                     Ngjf.write(f"{Nspin}\n")
#                     Nagjf.write(f"{Naspin}\n")
#                     Nmgjf.write(f"{Nmspin}\n")
#                 else:
#                     Ngjf.write(line)
#                     Nagjf.write(line)
#                     Nmgjf.write(line)


# '''
# @brief: read the SCF energy and eigenvalues of HOMO orbital

# @param: filename the log file

# @return: SCFenergy, eHOMO
# '''

# #* NOTICE: sometimes there will be not one SCF iterations in a log file,
# #* such as stable=opt, and we take the newest values
# def g16read(filename):
#     success = False

#     with open(filename, "r") as log:
#         for line in log:
#             if "SCF Done" in line:
#                 templine = line.split()
#                 SCFenergy = float(templine[4])
#             if "Alpha virt. eigenvalues" in line and "Alpha  occ. eigenvalues" in prev_line:
#                 templine = prev_line.split()
#                 eHOMO = float(templine[-1])
#             #* check if succeed
#             if "Normal termination" in line:
#                 success = True
#             prev_line = line    #! prev_line should be updated in the end of line

#         if not success:
#             raise Exception("Error: Gaussian doesn't finish correctly. (Does not finished normally.)")
    
#     return SCFenergy, eHOMO

In [24]:
# '''
# @brief: read the energy from the Gaussian output file, as a function y(x)

# @param wpara: parameter of functional LC-wPBE, as x
# @return: J2, as y

# @global: Nspin, Naspin, Nmspin
# '''

# def calIP(wpara):
#     global Nspin, Naspin, Nmspin

#     g16input(Nspin, Naspin, Nmspin, wpara)
#     #* run g16
#     #! remove the module load command later
#     exitcode1 = os.system("module load Gaussian16-B01;g16 < N.gjf > N.log")
#     exitcode2 = os.system("module load Gaussian16-B01;g16 < N+1.gjf > N+1.log")
#     exitcode3 = os.system("module load Gaussian16-B01;g16 < N-1.gjf > N-1.log")

#     #* check the exit code (non-zero means error)
#     if exitcode1 or exitcode2 or exitcode3:
#         print("Error in Gaussian16")
#         raise Exception("Error: Gaussian doesn't finish correctly. (exitcode does not equal to 0)")

#     #* store the output files
#     tempstr = f"{int(round(wpara,4)*10000):05}" 
#     os.system(f"mkdir {tempstr}")
#     os.system(f"cp *.log {tempstr}")

#     #* extract the energy
#     EN, eN = g16read("N.log")
#     ENa, eNa = g16read("N+1.log")
#     ENm, eNm = g16read("N-1.log")

#     JN = abs(eN + ENm - EN)
#     JNa = abs(eNa + EN - ENa)
#     J = JN + JNa
#     J2 = JN**2 + JNa**2

#     return J, J2

# '''
# @brief: point to the function calIP, only return J2
# #! Here wpara -> round(wpara, 4)

# @param wpara: parameter of functional LC-wPBE
# '''
# def funJ2(wpara):
#     global wparaList, J2List, JList, Nspin, Naspin, Nmspin
#     if round(wpara, 4) in wparaList:
#         #print(f"Already calculated.{round(wpara, 4)} {wparaList}")
#         return J2List[wparaList.index(wpara)]
#     J, J2 = calIP(wpara)
#     wparaList.append(round(wpara, 4))
#     JList.append(J)
#     J2List.append(J2)
#     return J2

In [107]:
from fileIO import fileClean
fileClean()

rm: cannot remove '06000': No such file or directory
rm: cannot remove '03899': No such file or directory
rm: cannot remove '02601': No such file or directory
rm: cannot remove '04688': No such file or directory
rm: cannot remove '04464': No such file or directory
rm: cannot remove '04413': No such file or directory
rm: cannot remove '04392': No such file or directory
rm: cannot remove '04390': No such file or directory
rm: cannot remove '04203': No such file or directory
rm: cannot remove '04319': No such file or directory
rm: cannot remove '04363': No such file or directory
rm: cannot remove '04380': No such file or directory
rm: cannot remove '04388': No such file or directory


In [53]:
wparaList = []
J2List = []
JList = []

chargeList = []
chargeSpinList = []

UDFT = []

oldchkBool = False
restartBool = False
cleanBool = False
#!NOTICE: N in line -> N+1 and N-1 also satisfy the condition
#!NOTICE: so it should be "N:" in line

#! global variables:
# chargeList, chargeSpinList, xmin, xmax, xguess, tol, cleanBool, restartBool, oldchkBool, wparaList, J2List, JList

readCharge = False
tmplist2 = []
with open("input", "r") as input:
    for line in input:
        if "restart" in line:
            restartBool = True
        
        if "oldchk" in line:
            oldchkBool = True

        if "clean" in line:
            cleanBool = True    

        if "orbit" in line:
            templine = line.strip().split()
            for i in range(1,len(templine)):
                tcharge1, tcharge2 = templine[i].split(",")
                #*there should be one extra file (N-1)
                tmplist = list(range(int(tcharge1)-1, int(tcharge2)+1))
                chargeList.extend(tmplist)
            #*remove the duplicate
            chargeList = list(set(chargeList))
            chargeList.sort()

        #* turn on the readCharge flag
        if "charge spin start" in line:
            readCharge = True
            continue

        if "charge spin end" in line:
            readCharge = False

        #* finish reading the charge spin

        if readCharge:
            tmplist1 = []
            templine = line.strip().split()
            #* templine: [charge in chargeList, charge, spin]
            for i in range(3):
                tmplist1.append(int(templine[i]))
            #* append the last element (may be 'U' or just duplicate of the charge spin)
            tmplist1.append(templine[-1])
            tmplist2.append(tmplist1)

            #* sort the charge spin list, by charge
            tmplist2 = sorted(tmplist2, key = lambda x: x[0])

        if "xmin" in line:
            templine = line.strip().split()
            xmin = float(templine[-1])
        if "xmax" in line:
            templine = line.strip().split()
            xmax = float(templine[-1])
        if "xguess" in line:
            templine = line.strip().split()
            xguess = float(templine[-1])
        if "tolerance" in line:
            templine = line.strip().split()
            tol = float(templine[-1])
    

#* check if the charge spin is complete
if len(tmplist2) != len(chargeList):
    raise Exception("Error: charge spin is not complete.")
#* convert the charge spin list to string
for i in range(len(tmplist2)):
    tmpstr = f"{tmplist2[i][1]} {tmplist2[i][2]}"
    if tmplist2[i][-1] == "U":
        UDFT.append(True)
    else:
        UDFT.append(False)
    chargeSpinList.append(tmpstr)

In [105]:
with open("output.txt", "a") as foutput:
    foutput.write("chargeList:\n")
    foutput.flush()
    

In [66]:
print(tempIop1)
print(ttt)

['#guess=read', 'stable=opt', 'ULC-wPBE']
#guess=read stable=opt ULC-wPBE/6-311+G** opt freq


In [81]:
import numpy as np

beta = np.zeros((3,3,3))
temp_beta = np.zeros((3,3,3))

In [101]:
#* input is XXX XXY XYY YYY XXZ XYZ YYZ XZZ YZZ ZZZ
ccsd = """
XXX=       -150.20500
XXY=          3.12910
XYY=         66.98390
YYY=        -99.80810
XXZ=          3.31752
XYZ=          1.76751
YYZ=          0.25054
XZZ=         66.74340
YZZ=         98.96630
ZZZ=          0.00000
"""
cam = """
 XXX=       -127.52800
 XXY=         -0.00293
 XYY=         63.76490
 YYY=        -90.16700
 XXZ=          0.00000
 XYZ=          0.00000
 YYZ=          0.00000
 XZZ=         63.75850
 YZZ=         90.16990
 ZZZ=          0.00000
"""
mp2 = """
 XXX=       -194.58000
 XXY=         -0.00788
 XYY=         97.28350
 YYY=       -137.56900
 XXZ=          0.00000
 XYZ=          0.00000
 YYZ=          0.00000
 XZZ=         97.27890
 YZZ=        137.58000
 ZZZ=          0.00000
"""

w0pbe = """
 XXX=        -99.28240
 XXY=         -0.00671
 XYY=         49.64040
 YYY=        -70.19600
 XXZ=          0.00000
 XYZ=          0.00000
 YYZ=          0.00000
 XZZ=         49.65120
 YZZ=         70.21150
 ZZZ=          0.00000
"""

wpbe = """
 XXX=        -103.928
 XXY=         -0.02928
 XYY=         51.9619
 YYY=        -73.4829
 XXZ=          0.00000
 XYZ=          0.00000
 YYZ=          0.00000
 XZZ=         51.9616
 YZZ=         73.4802
 ZZZ=          0.00000
"""

In [90]:
def calAvBeta(functional):

    lines = functional.split("\n")

    beta_static = []

    icount = 0
    for line in lines:
        icount += 1
        #* skip the first and last line
        if icount == 1 or icount == len(lines):
            continue
        beta_static.append(float(line.split("=")[1]))

    temp_beta[0,0,0] = beta_static[0]
    temp_beta[0,0,1] = beta_static[1]
    temp_beta[0,1,1] = beta_static[2]
    temp_beta[1,1,1] = beta_static[3]
    temp_beta[0,0,2] = beta_static[4]
    temp_beta[0,1,2] = beta_static[5]
    temp_beta[1,1,2] = beta_static[6]
    temp_beta[0,2,2] = beta_static[7]
    temp_beta[1,2,2] = beta_static[8]
    temp_beta[2,2,2] = beta_static[9]

    #* symmetrize the beta tensor
    for i in range(3):
        for j in range(3):
            for k in range(3):
                #* index of temp_beta are sorted
                itmp, jtmp, ktmp = sorted([i,j,k])

                beta[i,j,k] = temp_beta[itmp,jtmp,ktmp]
                beta[i,k,j] = temp_beta[itmp,jtmp,ktmp]
                beta[j,i,k] = temp_beta[itmp,jtmp,ktmp]
                beta[j,k,i] = temp_beta[itmp,jtmp,ktmp]
                beta[k,i,j] = temp_beta[itmp,jtmp,ktmp]
                beta[k,j,i] = temp_beta[itmp,jtmp,ktmp]


    sum1 = 0.0  # sum_i beta_iii**2
    sum2 = 0.0  # sum_i!=j beta_iii*beta_ijj
    sum3 = 0.0  # sum_i!=j beta_ijj**2
    sum4 = 0.0  # sum_i!=j!=k beta_ijj*beta_ikk
    sum5 = 0.0  # sum_i!=j!=k beta_ijk**2

    for i in range(3):
        for j in range(3):
            for k in range(3):
                if i == j == k:
                    sum1 += beta[i,i,i]**2
                elif j == k and i != j:
                    sum2 += beta[i,i,i]*beta[i,j,j]
                    sum3 += beta[i,j,j]**2
                elif i != j != k:
                    sum4 += beta[i,j,j]*beta[i,k,k]
                    sum5 += beta[i,j,k]**2

    ZZZ = 1.0/7.0 * sum1 + 6.0/35.0 * sum2 + 9.0/35.0 * sum3 + 3.0/35.0 * sum4 + 2.0/35.0 * sum5
    ZXX = 1.0/35.0 * sum1 - 2.0/105.0 * sum2 + 11.0/105.0 * sum3 - 1.0/105.0 * sum4 + 4.0/105.0 * sum5

    return ZZZ, ZXX

In [97]:
ZZZ, ZXX = calAvBeta(ccsd)

print(f"ZZZ: {ZZZ}")
print(f"ZXX: {ZXX}")
print(f"DR {ZZZ/ZXX}")

ZZZ: 3576.475729172228
ZXX: 4383.44912601458
DR 0.8159044684576847


In [98]:
ZZZ, ZXX = calAvBeta(cam)

print(f"ZZZ: {ZZZ}")
print(f"ZXX: {ZXX}")
print(f"DR {ZZZ/ZXX}")

ZZZ: 3020.058478364339
ZXX: 3639.5649401114997
DR 0.8297855727426087


In [99]:
ZZZ, ZXX = calAvBeta(w0pbe)

print(f"ZZZ: {ZZZ}")
print(f"ZXX: {ZXX}")
print(f"DR {ZZZ/ZXX}")

ZZZ: 1830.956053262431
ZXX: 2206.370943826681
DR 0.8298496036604168


In [102]:
ZZZ, ZXX = calAvBeta(wpbe)

print(f"ZZZ: {ZZZ}")
print(f"ZXX: {ZXX}")
print(f"DR {ZZZ/ZXX}")

ZZZ: 2005.7940266901253
ZXX: 2417.059156079924
DR 0.8298489599001774


In [100]:
ZZZ, ZXX = calAvBeta(mp2)

print(f"ZZZ: {ZZZ}")
print(f"ZXX: {ZXX}")
print(f"DR {ZZZ/ZXX}")

ZZZ: 7030.472649487953
ZXX: 8472.555796244344
DR 0.8297936087484219
